In [5]:
import cv2
import mediapipe as mp
from sklearn.neighbors import KNeighborsClassifier

gesture = {
    0:'fist', 1:'one', 2:'two', 3:'three', 4:'four', 5:'five',
    6:'six', 7:'rock', 8:'spiderman', 9:'yeah', 10:'ok',
}
rps_gesture = {0:'rock',5:'paper',9:'scissors'}

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5, 
    # 손의 위치 찾기
    min_tracking_confidence=0.5) 
#제스쳐 인식 모델 학습하기

import numpy as np
file = np.genfromtxt('gesture_train.csv',delimiter=',')
X_train = file[:,:-1].astype(np.float32)
y_train = file[:,-1].astype(np.float32)
knn= KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
        
    if not ret:
        print('비디오 읽기 실패 또는 모두 읽음')
        cap.release()
        cv2.destroyAllWindows()
        break
    
    img = cv2.flip(img,1)
    # 파이썬이 인식을 제대로 하게 하기 위해서
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)# 손가락에서 관절 위치 기억
    # 제대로 표현하기 위해서 다시 원상복귀
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    
    
   
    if result.multi_hand_landmarks is not None:
        # result : 21개의 관절, 각각의 3개 좌표
        rsp_result = []
        for res in result.multi_hand_landmarks:
            # res : 1개의 관절 3개의 좌표
            joint = np.zeros((21,3)) # 21개의 관절, 3개의 좌표를 담을 변수
            for j , lm in enumerate(res.landmark):
                joint[j] = [lm.x , lm.y , lm.z]
                # 1개의 관절값을 집어넣기
            
            # joint들로 관절 사이 직선 값 구하기 > 각도 구하기
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v 유클리디안 길이
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            
            # 관절값으로 관절 각도(radian) 구하기
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
            
            # radian 각도를 degree 각도로 변경하기
            angle = np.degrees(angle)
            
            # 학습할 형태로 변경
            X_test = np.array([angle],dtype=np.float32)
            
            # 제스쳐 인식시키기 > 0~10 까지의 행동을 숫자(실수)로 표현
            results = knn.predict(X_test)
            
            idx = int(results)
            
            # 제스쳐 인식되면 표시하기
#             cv2.putText(img,text=gesture[idx].upper(),
#                         org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)),
#                         fontFace=cv2.FONT_HERSHEY_SIMPLEX,
#                         fontScale=1, color=(255, 255, 255), thickness=2)
        
            # rsp만 표시하기
            if idx in rps_gesture.keys():
                org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20))
                cv2.putText(img,text=rps_gesture[idx].upper(),
                        org = org,
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=1, 
                        color=(255, 255, 255), 
                        thickness=2)
                
                rsp_result.append({
                    'rsp' : rps_gesture[idx],
                    'org' : org
                })
            mp_drawing.draw_landmarks(img,res,mp_hands.HAND_CONNECTIONS)

            # 승패 판별
            if len(rsp_result)==2:
                winner = None
                text = ''
                
                # 가위바위보 결과 계산
                if rsp_result[0]['rsp']=='rock':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Tie'
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Paper wins'  ; winner = 1
                    elif rsp_result[1]['rsp']=='scissors': text = 'Rock wins'   ; winner = 0
                elif rsp_result[0]['rsp']=='paper':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Paper wins'  ; winner = 0
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Tie'
                    elif rsp_result[1]['rsp']=='scissors': text = 'Scissors wins'; winner = 1
                elif rsp_result[0]['rsp']=='scissors':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Rock wins'   ; winner = 1
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Scissors wins'; winner = 0
                    elif rsp_result[1]['rsp']=='scissors': text = 'Tie'
                
                if winner is not None: # 승자가 나왔을 때
                    cv2.putText(img, text='Winner', 
                                org=(rsp_result[winner]['org'][0], 
                                     rsp_result[winner]['org'][1] + 70), 
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                fontScale=2, color=(0, 255, 0), 
                                thickness=3)
                cv2.putText(img, 
                            text=text, 
                            org=(int(img.shape[1] / 3), 100), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                            fontScale=2, color=(0, 0, 255), 
                            thickness=3) # 어떤 행동이 이겼는지 표현
                
    cv2.imshow('Game',img)
    k = cv2.waitKey(33)
    
    if k==49:
        cap.release()
        cv2.destroyAllWindows()
        break